In [15]:
import numpy as np
from scipy.stats import entropy

In [2]:
def parse_input(file):

    with open(file) as file_in:
        rows = file_in.read().splitlines()

    robots_param = []
    for row in rows:
        pos, velocity = row.split(' ')
        pos_y, pos_x = pos.split('=')[1].split(',')
        vel_y, vel_x = velocity.split('=')[1].split(',')
        robots_param.append([(int(pos_x), int(pos_y)), (int(vel_x), int(vel_y))])

    return robots_param

In [3]:
def get_new_pos(n_seconds, pos_x, pos_y, vel_x, vel_y, n_rows, n_cols):
    new_pos_x = (pos_x + n_seconds * vel_x) % n_rows
    new_pos_y = (pos_y + n_seconds * vel_y) % n_cols
    return new_pos_x, new_pos_y

In [4]:
def count_robots_in_quadrants(pos_robots, n_rows, n_cols):
    med_x, med_y = n_rows // 2, n_cols // 2
    nw, ne, sw, se = 0, 0, 0, 0
    for pos_x, pos_y in pos_robots:
        if pos_x < med_x and pos_y < med_y:
            nw += 1
        elif pos_x < med_x and pos_y > med_y:
            ne += 1
        elif pos_x > med_x and pos_y < med_y:
            sw += 1
        elif pos_x > med_x and pos_y > med_y:
            se += 1
    return nw, ne, sw, se

In [5]:
def main1(file, n_rows, n_cols, n_seconds):
    robots_param = parse_input(file)

    robots_new_pos = []
    for robot in robots_param:
        (pos_x, pos_y), (vel_x, vel_y) = robot
        robots_new_pos.append(get_new_pos(n_seconds, pos_x, pos_y, vel_x, vel_y, n_rows, n_cols))

    nw, ne, sw, se = count_robots_in_quadrants(robots_new_pos, n_rows, n_cols)

    return nw * ne * sw * se

In [6]:
assert main1('example.txt', n_rows=7, n_cols=11, n_seconds=100) == 12

In [7]:
main1('input.txt', n_rows=103, n_cols=101, n_seconds=100)

224357412

In [11]:
def plot_robots(robots, n_rows, n_cols, path):
    plot = np.full((n_rows, n_cols), '.')
    for pos_x, pos_y in robots:
        plot[pos_x, pos_y] = 'R'
    with open(path, 'w') as file_out:
        for row in plot:
            file_out.write("".join(row) + "\n")

In [25]:
robots_param = parse_input('input.txt')
n_rows, n_cols = 103, 101

test = []
for n_sec in range(10000):
    robots_new_pos = []
    for robot in robots_param:
        (pos_x, pos_y), (vel_x, vel_y) = robot
        robots_new_pos.append(get_new_pos(n_sec, pos_x, pos_y, vel_x, vel_y, n_rows, n_cols))
    nw, ne, sw, se = count_robots_in_quadrants(robots_new_pos, n_rows, n_cols)
    n_total = sum([nw, ne, sw, se])
    distrib = [nw / n_total, ne / n_total, sw / n_total, se / n_total]
    if entropy(distrib) < 1.35:
        plot_robots(robots_new_pos, n_rows, n_cols, path=f'output/{n_sec}.txt')